In [ ]:
import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
# Get performer data from Stash app
performers = stash.find_performers()

# Convert the list of performers to a pandas DataFrame
df_performers = pd.DataFrame(performers)

# Filter the performers
df_females = df_performers[df_performers['gender'].isin(['FEMALE', 'TRANSGENDER_FEMALE'])].copy()

# Calculations
current_date = pd.Timestamp.now()
df_females['birthdate'] = pd.to_datetime(df_females['birthdate'])
df_females['age'] = df_females['birthdate'].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)))

df_females['cup_size'] = df_females['measurements'].str.extract(r'(\d+)([A-Z]+)')[1]

# Only select relevant columns
df_females = df_females[["id", "name", "birthdate", "age", "ethnicity", "country", "eye_color", "hair_color", "height_cm", "measurements", "cup_size", "fake_tits", "tags"]]

In [ ]:
df_b_cup_females = df_females[df_females["cup_size"].isin(["B"])]

b_cup_tag = stash.find_tag({ "name": "B cup" }, create=True)

for index, b_cup_female in df_b_cup_females.iterrows():
    tags = [tag["id"] for tag in b_cup_female["tags"]]
    tags.append(b_cup_tag["id"])
    stash.update_performer({ "id": b_cup_female["id"], "tag_ids": tags })
    